# CS4220 Project 2 - Pathogen Detection

In this notebook, we give an example to show how to read and use the DNA read data. We will train one base line model, using `sklearn.LogisticRegression`, and use it to predict the pathogens in each patient's dataset.

## Related python packages

To get started (if you are using python), you may want to create a virtual python environment and install some packages. Here are some of the commands you might need:

```bash
conda create --name cs4220 python=3.8

# Activate the created virtual environment
conda activate cs4220

# Install jupyter notebook if you are using it
conda install -c anaconda ipykernel
python -m ipykernel install --user --name=cs4220
conda install -c anaconda jupyter

# Some common packages
conda install pandas                      # for reading csv
conda install scikit-learn                # for the logistic regression model
pip install torch                         # if you are using neural networks
conda install -c conda-forge matplotlib   # for plotting
conda install seaborn                     # also for plotting
pip install umap-learn[plot]              # plotting UMAP plots
conda install numpy                       # for many math/vectorized operations
```

In [2]:
# import packages
import numpy as np
import pandas as pd
import timeit
import time
from sklearn.linear_model import LogisticRegression
from sklearn import preprocessing
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns
import pickle as pkl
from itertools import combinations
import glob 
import csv
from joblib import dump, load

## Load Groups and Assigning Groups
First, we create labels with the groups decided by Jaccard index.

In [3]:
with open("groups"+str(8)+".csv", "r") as file:
    reader = csv.reader(file)
    groups_loaded = [row for row in reader]

groups_loaded

[['corynebacterium_diphtheriae',
  'corynebacterium_striatum',
  'corynebacterium_ulcerans'],
 ['acinetobacter_baumannii',
  'enterococcus_faecium',
  'legionella_pneumophila',
  'listeria_monocytogenes',
  'staphylococcus_aureus',
  'staphylococcus_epidermidis',
  'staphylococcus_haemolyticus',
  'staphylococcus_pseudintermedius',
  'staphylococcus_pyogenes',
  'streptococcus_agalactiae',
  'streptococcus_anginosus',
  'streptococcus_mitis',
  'streptococcus_pneumoniae',
  'streptococcus_salivarius',
  'streptococcus_suis'],
 ['mycobacterium_tuberculosis',
  'mycobacterium_ulcerans',
  'pseudomonas_aeruginosa',
  'stenotrophomonas_maltophilia'],
 ['escherichia_coli',
  'klebsiella_michiganensis',
  'klebsiella_pneumoniae',
  'salmonella_enterica_typhimurium',
  'serratia_liquefaciens',
  'vibrio_parahaemolyticus',
  'yersinia_enterocolitica'],
 ['neisseria_gonorrhoeae', 'neisseria_lactamica']]

In [4]:
#Make labels
label_df = pd.read_csv('./training_data/train_labels.csv')
label_df['species_name'].value_counts()

species_name
homo_sapiens                       400027
burkholderia_pseudomallei            3540
pseudomonas_aeruginosa               3111
mycobacterium_ulcerans               3024
klebsiella_michiganensis             3010
bacillus_cereus                      2889
klebsiella_pneumoniae                2799
escherichia_coli                     2760
serratia_liquefaciens                2685
vibrio_parahaemolyticus              2572
stenotrophomonas_maltophilia         2524
salmonella_enterica_typhimurium      2491
yersinia_enterocolitica              2271
mycobacterium_tuberculosis           2173
clostridioides_difficile             2020
acinetobacter_baumannii              2005
legionella_pneumophila               1674
listeria_monocytogenes               1487
enterococcus_faecium                 1467
corynebacterium_striatum             1437
staphylococcus_aureus                1392
staphylococcus_haemolyticus          1298
staphylococcus_pseudintermedius      1254
corynebacterium_dipht

In [5]:
def assign_group(species, species_list, group_id):
    if species in species_list:
        return group_id
    return species

#TO APPEND TO
label_groups = {}
species_to_keep = {}
master_labels = label_df.copy()
grouped_labels = label_df.copy()

#FOR GROUP 1
species_to_keep[1] = ['corynebacterium_diphtheriae',
'corynebacterium_striatum',
'corynebacterium_ulcerans']

#FOR GROUP 2
species_to_keep[2] = ['acinetobacter_baumannii',
'enterococcus_faecium',
'legionella_pneumophila',
'listeria_monocytogenes',
'staphylococcus_aureus',
'staphylococcus_epidermidis',
'staphylococcus_haemolyticus',
'staphylococcus_pseudintermedius',
'staphylococcus_pyogenes',
'streptococcus_agalactiae',
'streptococcus_anginosus',
'streptococcus_mitis',
'streptococcus_pneumoniae',
'streptococcus_salivarius',
'streptococcus_suis']

#FOR GROUP 3
species_to_keep[3] = ['mycobacterium_tuberculosis',
  'mycobacterium_ulcerans',
  'pseudomonas_aeruginosa',
  'stenotrophomonas_maltophilia']

#FOR GROUP 4
species_to_keep[4] = ['escherichia_coli',
  'klebsiella_michiganensis',
  'klebsiella_pneumoniae',
  'salmonella_enterica_typhimurium',
  'serratia_liquefaciens',
  'vibrio_parahaemolyticus',
  'yersinia_enterocolitica']

#FOR GROUP 5
species_to_keep[5] = ['neisseria_gonorrhoeae', 'neisseria_lactamica']

species_to_keep[6] = ['homo_sapiens']

#REPLACING OUTGROUP SPECIES W 'Other'
for i in range(1,7):
    label_groups[i] = label_df.copy()

    #keep label if in species to keep
    label_groups[i]['species_name'] = label_groups[i]['species_name'].apply(lambda x: x if x in species_to_keep[i] else 'Other')

    #replace w group # if in species to keep 
    grouped_labels['species_name'] = grouped_labels['species_name'].apply(lambda x: assign_group(x, species_to_keep[i], str(i)))


In [6]:
#HUMAN
lehumans = preprocessing.LabelEncoder()
lehumans.fit(label_groups[6]['species_name'].unique())
y_index = lehumans.transform(label_groups[6]['species_name'].values)
label_groups[6]['labels'] = y_index

#GROUPED, FIRST LAYER
le = preprocessing.LabelEncoder()
le.fit(grouped_labels['species_name'].unique())
y_index = le.transform(grouped_labels['species_name'].values)
grouped_labels['labels'] = y_index

#MASTER GROUPS
masle = preprocessing.LabelEncoder()
masle.fit(master_labels['species_name'].unique())
y_index = masle.transform(master_labels['species_name'].values)
master_labels['labels'] = y_index

#GROUP 1
le1 = preprocessing.LabelEncoder()
le1.fit(label_groups[1]['species_name'].unique())
y_index = le1.transform(label_groups[1]['species_name'].values)
label_groups[1]['labels'] = y_index

#GROUP 2
le2 = preprocessing.LabelEncoder()
le2.fit(label_groups[2]['species_name'].unique())
y_index = le2.transform(label_groups[2]['species_name'].values)
label_groups[2]['labels'] = y_index

#GROUP 3
le3 = preprocessing.LabelEncoder()
le3.fit(label_groups[3]['species_name'].unique())
y_index = le3.transform(label_groups[3]['species_name'].values)
label_groups[3]['labels'] = y_index

#GROUP 4
le4 = preprocessing.LabelEncoder()
le4.fit(label_groups[4]['species_name'].unique())
y_index = le4.transform(label_groups[4]['species_name'].values)
label_groups[4]['labels'] = y_index

#GROUP 5
le5 = preprocessing.LabelEncoder()
le5.fit(label_groups[5]['species_name'].unique())
y_index = le5.transform(label_groups[5]['species_name'].values)
label_groups[5]['labels'] = y_index

In [7]:
samples_index = master_labels.groupby('labels').sample(800).index
samples_index

Index([   986,   1701,   1328,   1096,    761,    883,    529,    976,   1417,
         1510,
       ...
       464974, 464877, 463338, 464316, 463939, 464375, 464991, 464632, 463323,
       464859],
      dtype='int64', length=29600)

In this project, we try to use canonical $k$-mer profiles to represent each read in the input. Here, we are using $k=6$ and consequently 2081 features (including 1 feature `IGNORE` for ambiguous $k$-mer) for each read. Read [this reference](https://bioinfologics.github.io/post/2018/09/17/k-mer-counting-part-i-introduction/) for more information.

To help you save time, we implemented a utility class `CS4220Dataset` that can
- take in raw reads as input (`.fasta`, `.fa` files), and turn them into $k$-mer profiles, or
- take in $k$-mer profile as input (`.npy` files),
- allow you to sample data or create $k$-mer profile on the fly (during training) to save memory.

In [8]:
# Load dictionary that maps k-mer to their corresponding index.
# A k-mer and its reverse complement are mapped to the same index.
# We use k=6 here as an example.

import json

with open("./training_data/8-mers.json", 'r') as dict_file:
    canonical_kmer_dict = json.load(dict_file)

In [9]:
def sequence_to_kmer_profile(sequence : str, k : int = 8):
    """
    Return the k-mer profile of the input sequence (string)
    """
    res = np.zeros(len(set(canonical_kmer_dict.values())))
    for i in range(len(sequence) - k + 1):
        k_mer = sequence[i:i + k]
        if k_mer in canonical_kmer_dict:
            res[canonical_kmer_dict[k_mer]] += 1
        else:
            res[-1] += 1

    res /= np.sum(res)
    return res

In [10]:
import torch
from tqdm import tqdm
from torch.utils.data import Dataset, DataLoader

class CS4220Dataset(Dataset):
    def __init__(self, data_file, label_df=None, k=8, samples_index=None, kmer_profile_on_the_fly=False, dtype=np.float32):
        """
        Dataset class to load large CS4220 sequence database.

        Args:
            - data_file (`str`): Can either be a *.fasta file if the input is raw reads, or *.npy file
                                 if the input is k-mer profile.
            - label_df (`pd.DataFrame` or `None`): A dataframe with "labels" column indicating the label
                                                   of the data (must match with data_file), or `None` if there is
                                                   no label (in the case of test sets).
            - k (`int`): The lengt of k-mer. We use 6 in this project.
            - samples_index (`List` or `None`): list of indices of data we sample from the data file. You
                                                can use this if the dataset is very large and can't fit in memory.
                                                set this to `None` if you want to use all the data.
            - kmer_profile_on_the_fly (`bool`): If input data_file is raw reads and this set to `True`,
                                                we will build k-mer profile on the fly. This is helpful if you want to
                                                alter the input sequences during training, or the k-mer profile can't fit in memory.
                                                Otherwise, we build k-mer profile in advance, which will speed up the
                                                training process.
            - dtype: type to store the k-mer profile. You may use, for example, `np.float32` for better precision,
                     or `np.float16` for smaller memory usage. If loaded from ".npy" file, it is always `np.float16`.
        """
        self.data_file = data_file

        if ".fasta" in data_file or ".fa" in data_file or ".fna" in data_file:
            self.is_raw_reads = True
        elif ".npy" in data_file:
            self.is_raw_reads = False
        else:
            raise TypeError(f"The input file must be either a fasta file containing raw reads (.fasta, .fa, .fna) or a numpy file containing k-mer profiles (.npy).")


        self.label_df = label_df
        self.kmer_profile_otf = kmer_profile_on_the_fly

        # k-mer length, set to be 6.
        self.k = k

        # the samples we take from the read dataset
        self.samples_index = samples_index

        self.dtype = dtype

        # Load the data and store in self.reads and self.labels
        self.X = []
        self.Y = []
        self._read_labels()
        self._read_data()


    def _read_labels(self):
        """
        Read the labels and record them in self.labels.
        """
        if self.label_df is None:
            self.Y = None
        elif self.samples_index is None:
            # Load the whole dataset
            self.Y = list(self.label_df["labels"])
        else:
            # Load only the data corresponding to the sampled index
            self.Y = list(self.label_df.iloc[self.samples_index]["labels"])


    def _read_data(self):
        if self.is_raw_reads:
            # Read the fasta file
            with open(self.data_file, 'r') as fasta_file:
                lines = fasta_file.readlines()

            read_range = self.samples_index if self.samples_index is not None else range(int(len(lines) / 2))
            if not self.kmer_profile_otf:
                self.X = np.zeros(
                    (len(read_range), len(set(canonical_kmer_dict.values()))),
                    dtype=self.dtype
                )

            for i, j in enumerate(tqdm(read_range, desc=f"Parsing fasta file {self.data_file}")):
                read = lines[j * 2 + 1].strip()
                if self.kmer_profile_otf:
                    # If chose to do k-mer profiling on the fly, simply store the reads
                    self.X.append(read)
                else:
                    # Otherwise, do k-mer profiling during training/testing, cost more time during training/testing
                    self.X[i, :] = sequence_to_kmer_profile(read, self.k)
        else:
            # Read the .npy file, and load the numpy matrix
            # Each row corresponds to a read, and each column corresponds to a k-mer (see training_data/6-mers.txt).
            self.X = np.load(self.data_file)
            if self.samples_index is not None:
                self.X = self.X[self.samples_index, :]

    def __len__(self):
        return len(self.X)

    def __getitem__(self, idx):
        """
        If you are using pytorch, this function helps taking data points during each epoch
        of your training.
        """
        x = self.X[idx]
        if self.kmer_profile_otf:
            read_tensor = torch.tensor(sequence_to_kmer_profile(x, self.k, self.count), dtype=self.dtype)
        else:
            read_tensor = torch.tensor(x)

        label = self.Y[idx] if self.Y is not None else None
        return read_tensor, label




The data and labels are then accessible via `sampled_dataset.X` and `sampled_dataset.Y`.

### TRAINING

# GRID SEARCH + TRAINING FOR GROUPS

# TRAINING MODEL FOR GROUP CLASSIFICATION 
Train model, then get sub-dataframes to feed into further models

No backtracking methods in mind, but later models might assign smth as Other. Others are human (decide if we change this)
_ improvement: fallback classifier

In [11]:
from sklearn.ensemble import RandomForestClassifier
input_file_path = './training_data/train_raw_reads.fasta'

## RANDOM FOREST

### HUMANS

In [12]:
sampled_dataset_humans = CS4220Dataset(input_file_path, label_groups[6], k=8, samples_index=samples_index)


Parsing fasta file ./training_data/train_raw_reads.fasta: 100%|██████████| 29600/29600 [03:36<00:00, 136.79it/s]


In [13]:
rf = RandomForestClassifier(
        n_estimators=100,
        max_depth=None,
        random_state=42,
        n_jobs=-1)

rf.fit(sampled_dataset_humans.X, sampled_dataset_humans.Y)

RandomForestClassifier(n_jobs=-1, random_state=42)

In [14]:
from joblib import dump, load
dump(rf, 'models/rf_humans.joblib')

['models/rf_humans.joblib']

### GROUPS

In [15]:
#removes humans so only indices w nonhumans are picked
nonhumans_master = master_labels[master_labels['species_name'] != 'homo_sapiens']
nonhumans_index = nonhumans_master.groupby('labels').sample(800).index


In [16]:
#sample data set
sampled_dataset_nonhumans_groups = CS4220Dataset(input_file_path, grouped_labels, k=8, samples_index=nonhumans_index)


Parsing fasta file ./training_data/train_raw_reads.fasta: 100%|██████████| 28800/28800 [03:21<00:00, 142.71it/s]


In [17]:
rf_nonhumans_groups = RandomForestClassifier(
        n_estimators=100,
        max_depth=None,
        random_state=42,
        n_jobs=-1)

rf_nonhumans_groups.fit(sampled_dataset_nonhumans_groups.X, sampled_dataset_nonhumans_groups.Y)

RandomForestClassifier(n_jobs=-1, random_state=42)

In [18]:
dump(rf_nonhumans_groups, 'models/rf_nonhumans_into_groups.joblib')

['models/rf_nonhumans_into_groups.joblib']

### GROUPS 1 - 5

In [19]:
only = {}
only_index = {}
sampled_dataset_only = {}

for i in range(1, 6):
    #removes non-group 1 so only indices w group1 are picked
    only[i] = label_groups[i][label_groups[i]['species_name'] != 'Other']
    only_index[i] = only[i].groupby('labels').sample(800).index

    #sample data set
    sampled_dataset_only[i] = CS4220Dataset(input_file_path, label_groups[i], k=8, samples_index=only_index[i])


Parsing fasta file ./training_data/train_raw_reads.fasta: 100%|██████████| 2400/2400 [00:18<00:00, 132.36it/s]
Parsing fasta file ./training_data/train_raw_reads.fasta: 100%|██████████| 12000/12000 [01:15<00:00, 159.24it/s]
Parsing fasta file ./training_data/train_raw_reads.fasta: 100%|██████████| 3200/3200 [00:21<00:00, 149.56it/s]
Parsing fasta file ./training_data/train_raw_reads.fasta: 100%|██████████| 5600/5600 [00:36<00:00, 152.12it/s]
Parsing fasta file ./training_data/train_raw_reads.fasta: 100%|██████████| 1600/1600 [00:09<00:00, 167.79it/s]


In [20]:
rf_only = {}

#make rfs for the groups
for i in range(1, 6):
    rf_only[i] = RandomForestClassifier(
        n_estimators=100,
        max_depth=None,
        random_state=42,
        n_jobs=-1)
    
    rf_only[i].fit(sampled_dataset_only[i].X, sampled_dataset_only[i].Y)

In [21]:
#dump!!! lets never run these again. 
dump(rf, 'models/rf_humans.joblib')
dump(rf_nonhumans_groups, 'models/rf_nonhumans_into_groups.joblib')
dump(rf_only[1], 'models/rf_group1.joblib')
dump(rf_only[2], 'models/rf_group2.joblib')
dump(rf_only[3], 'models/rf_group3.joblib')
dump(rf_only[4], 'models/rf_group4.joblib')
dump(rf_only[5], 'models/rf_group5.joblib')

['models/rf_group5.joblib']